In [1]:
import subprocess
import sys
import os
from datetime import datetime,timedelta,timezone
from typing import Optional,List,Tuple,Dict
from dataclasses import dataclass
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ccxt
import yfinance as yf

pd.options.display.float_format='{:.6f}'.format



In [12]:
# CELLULE 2 : CHARGEMENT GÉNÉRIQUE

def fetch_ohlcv(symbol, timeframe='1d', since='2017-01-01'):
    """Récupère l'OHLCV pour n'importe quel symbole"""
    print(f"--- Récupération des prix pour {symbol} ---")
    ex = ccxt.binance({'enableRateLimit': True})
    since_ms = int(pd.Timestamp(since, tz='UTC').timestamp() * 1000)
    all_ohlcv = []
    
    while True:
        try:
            ohlcv = ex.fetch_ohlcv(symbol, timeframe, since=since_ms, limit=1000)
            if not ohlcv: break
            all_ohlcv.extend(ohlcv)
            since_ms = ohlcv[-1][0] + 1
            if len(ohlcv) < 1000: break
        except Exception as e:
            print(f"Erreur fetch prix: {e}")
            break
            
    if not all_ohlcv:
        return pd.DataFrame()

    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df.set_index('timestamp', inplace=True)
    return df

def fetch_funding(symbol, limit=1000):
    """Construit automatiquement le ticker futures (ex: BTC/USDT -> BTC/USDT:USDT)"""
    # Construction du symbole futures standard Binance
    futures_symbol = f"{symbol}:USDT" 
    print(f"--- Récupération du Funding Rate pour {futures_symbol} ---")
    
    ex = ccxt.binance({'enableRateLimit': True, 'options': {'defaultType': 'future'}})
    rates = []
    # On prend 4 ans d'historique max pour le funding
    since = int((datetime.now(tz=timezone.utc) - timedelta(days=365*4)).timestamp() * 1000)
    
    while True:
        try:
            batch = ex.fetch_funding_rate_history(futures_symbol, since=since, limit=limit)
            if not batch: break
            rates.extend(batch)
            since = batch[-1]['timestamp'] + 1
            if len(batch) < limit: break
        except Exception as e:
            # Certains coins n'ont pas de futures ou pas d'historique long
            print(f"Note: Funding rate non dispo ou erreur ({e})")
            break
            
    if not rates: 
        return pd.Series(dtype=float)
        
    df = pd.DataFrame(rates)
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df.set_index('datetime', inplace=True)
    return df['fundingRate']

In [13]:
def generate_pure_tsm_signals(prices: pd.Series, lookbacks: List[int]) -> pd.DataFrame:
    signals = pd.DataFrame(index=prices.index)
    for lb in lookbacks:
        raw_mom = prices.pct_change(lb)
        window_norm = 252
        rolling_mean = raw_mom.rolling(window_norm).mean()
        rolling_std = raw_mom.rolling(window_norm).std()
        z_mom = (raw_mom - rolling_mean) / rolling_std
        signals[f'tsm_{lb}'] = z_mom.clip(-3, 3)
    return signals.dropna()



In [14]:
class RollingPCAEngine:
    def __init__(self, window: int = 120):
        self.window = window
        
    def compute_signal(self, X: pd.DataFrame) -> pd.Series:
        rolling_mean = X.rolling(self.window).mean()
        rolling_std = X.rolling(self.window).std()
        X_norm = (X - rolling_mean) / rolling_std
        signal = X_norm.mean(axis=1)
        return signal

def run_pure_tsm_pca(prices: pd.Series, funding: Optional[pd.Series] = None, 
                     lookbacks: List[int] = [10, 30, 60, 90, 180, 365], 
                     pca_window: int = 120, target_vol: float = 0.90, 
                     max_leverage: float = 2.0, vol_floor: float = 0.20):
    
    prices = prices.dropna().sort_index()
    idx = prices.resample('W').last().index
    
    # 1. Signaux & PCA (Inchangé)
    df_signals = generate_pure_tsm_signals(prices, lookbacks)
    pca = RollingPCAEngine(window=pca_window)
    pca_sig_daily = pca.compute_signal(df_signals)
    pca_sig_daily = pca_sig_daily.ewm(span=3).mean()
    signal_w = pca_sig_daily.reindex(idx, method='ffill').shift(1).fillna(0.0)
    
    # 2. Volatility Targeting STANDARD (Académique)
    r = prices.pct_change().dropna()
    
    # --- CHANGEMENT ICI ---
    # On calcule l'écart-type standard sur TOUS les rendements.
    # Si le Bitcoin fait +10%, c'est considéré comme du risque.
    vol_total = r.ewm(span=30).std() * np.sqrt(365)
    
    # On échantillonne en Hebdo
    vol_w = vol_total.reindex(idx, method='ffill').clip(lower=vol_floor)
    
    # Calcul du Levier (Target / Total Vol)
    leverage = (target_vol / vol_w).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    leverage = leverage.clip(0, max_leverage)
    
    # 3. Position & PnL
    direction = np.where(signal_w > 0, 1.0, 0.0)
    pos = pd.Series(direction, index=idx) * leverage
    
    df = pd.DataFrame(index=idx)
    df['asset_return'] = prices.reindex(idx, method='ffill').pct_change().fillna(0.0)
    df['position'] = pos.fillna(0.0)
    df['cost'] = df['position'].diff().abs().fillna(0.0) * (10.0 / 10000.0)
    
    if funding is not None:
        fund_w = funding.resample('W').sum().reindex(idx).fillna(0.0)
        df['fund_pnl'] = fund_w * df['position']
    else:
        df['fund_pnl'] = 0.0
        
    df['net_ret'] = df['position'] * df['asset_return'] - df['cost'] + df['fund_pnl']
    df['equity'] = (1 + df['net_ret']).cumprod()
    
    return df

In [ ]:

TICKER = 'BTC/USDT' 

df_asset = fetch_ohlcv(TICKER, since='2017-01-01')

if df_asset.empty:
    print("Pas de données trouvées. Vérifiez le ticker.")
else:
    print(f"{TICKER} Data: {df_asset.index.min()} -> {df_asset.index.max()}")

    try:
        fund = fetch_funding(TICKER)
        if not fund.empty:
            fund = fund.groupby(level=0).last()
        else:
            fund = None
    except:
        fund = None

    results = run_pure_tsm_pca(
        df_asset['close'], 
        funding=fund,
        lookbacks=[10, 21, 63, 126, 252],
        pca_window=120,
        target_vol=0.90,  
        max_leverage=2.0
    )
    print("Backtest terminé.")

--- Récupération des prix pour BTC/USDT ---
BTC/USDT Data: 2017-08-17 00:00:00+00:00 -> 2025-11-25 00:00:00+00:00
--- Récupération du Funding Rate pour BTC/USDT:USDT ---
Backtest terminé.


In [21]:
# Cell 4: Display (English Version - Clean Log Axis)

ann_ret = results['net_ret'].mean() * 52
ann_vol = results['net_ret'].std() * np.sqrt(52)
sharpe = ann_ret / ann_vol

print(f"Sharpe Ratio : {sharpe:.2f}")
print(f"Ann. Return  : {ann_ret*100:.1f}%")

# 1. Get start price for scaling
start_date = results.index[0]
start_price = df_btc['close'].reindex([start_date], method='nearest').iloc[0]

# 2. Conversion: Equity -> Price ($)
strategy_price_curve = results['equity'] * start_price
btc_price_curve = df_btc['close'].reindex(results.index, method='ffill')

fig = go.Figure()

# Strategy Curve
fig.add_trace(go.Scatter(
    x=results.index, 
    y=strategy_price_curve, 
    name='Strategy (Theoretical Value)', 
    line=dict(color='#00cc96', width=2)
))

# Bitcoin Curve
fig.add_trace(go.Scatter(
    x=results.index, 
    y=btc_price_curve, 
    name='Bitcoin Price', 
    line=dict(color='grey', dash='dot')
))

# --- CLEAN Y-AXIS CONFIGURATION ---
fig.update_layout(
    title=f'Price Comparison: Strategy vs Reality (Sharpe: {sharpe:.2f})', 
    template='plotly_dark',
    yaxis_title='Price ($) - Log Scale',
    yaxis=dict(
        type='log',
        # Force "array" mode to pick numbers ourselves
        tickmode='array',
        # Exact values we want to see
        tickvals=[3000, 5000, 10000, 20000, 50000, 100000, 200000, 300000],
        # What is written on screen
        ticktext=['$3k', '$5k', '$10k', '$20k', '$50k', '$100k', '$200k', '$300k'],
        # Discrete gray grid
        gridcolor='rgba(128,128,128,0.2)'
    )
)

fig.show()

Sharpe Ratio : 1.24
Ann. Return  : 67.6%


In [7]:
print(f"Average Leverage Used: {results['position'].mean():.2f}x")
print(f"Time in Market: {(results['position'] > 0).mean()*100:.1f}%")

Average Leverage Used: 0.64x
Time in Market: 39.5%


In [8]:
# 1. Calculate Drawdown
# Current Equity / Highest Equity ever reached - 1
drawdown = (results['equity'] / results['equity'].cummax()) - 1

# 2. Plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=drawdown.index, 
    y=drawdown, 
    fill='tozeroy', # Red fill
    line=dict(color='#ff4d4d', width=1),
    name='Drawdown'
))

fig.update_layout(
    title=f'Underwater Plot (Max Drawdown: {drawdown.min():.2%})',
    template='plotly_dark',
    yaxis_tickformat='.1%', # Percentage format
    yaxis_title='Distance from Peak (%)'
)
fig.show()

In [9]:
benchmark_equity = (1 + results['asset_return']).cumprod()
alpha_curve = results['equity'] / benchmark_equity

fig_alpha = go.Figure()

fig_alpha.add_trace(go.Scatter(
    x=alpha_curve.index, 
    y=alpha_curve, 
    name='Alpha Generation', 
    line=dict(color='#FFD700', width=2), # Couleur Or pour l'Alpha
    fill='tozeroy',
    fillcolor='rgba(255, 215, 0, 0.1)'
))

fig_alpha.update_layout(
    title='Alpha Curve: Strategy Performance vs Bitcoin',
    template='plotly_dark',
    yaxis_title='Relative Strength (Strategy / BTC)',
    yaxis_tickformat='.2f'
)

fig_alpha.add_hline(y=1.0, line_dash="dash", line_color="gray")

fig_alpha.show()

In [10]:
weekly_rets = results['net_ret'].resample('W').sum()

win_rate = (weekly_rets > 0).mean()
avg_win = weekly_rets[weekly_rets > 0].mean()
avg_loss = weekly_rets[weekly_rets < 0].mean()
profit_factor = abs(weekly_rets[weekly_rets > 0].sum() / weekly_rets[weekly_rets < 0].sum())

print(f"Win Rate (Semaines Gagnantes) : {win_rate*100:.1f}%")
print(f"Gain Moyen : {avg_win*100:.2f}%  |  Perte Moyenne : {avg_loss*100:.2f}%")
print(f"Profit Factor : {profit_factor:.2f}")

Win Rate (Semaines Gagnantes) : 22.9%
Gain Moyen : 10.79%  |  Perte Moyenne : -5.14%
Profit Factor : 2.10


In [11]:
# Cell: 2025 Zoom (Year-to-Date)

# 1. Slice data starting from 2025
subset_2025 = results.loc['2025-01-01':].copy()

# 2. Rebase (Reset everything to 100% at the start)
# Otherwise, the strategy would start with its accumulated gains from 2017
subset_2025['strat_base100'] = subset_2025['equity'] / subset_2025['equity'].iloc[0]

# For Bitcoin, take the real price and rebase it as well
btc_prices_2025 = df_btc['close'].reindex(subset_2025.index, method='ffill')
subset_2025['btc_base100'] = btc_prices_2025 / btc_prices_2025.iloc[0]

# 3. 2025 Stats
ret_strat_2025 = (subset_2025['strat_base100'].iloc[-1] - 1) * 100
ret_btc_2025 = (subset_2025['btc_base100'].iloc[-1] - 1) * 100

print(f"--- 2025 PERFORMANCE (YTD) ---")
print(f"Strategy  : {ret_strat_2025:+.1f}%")
print(f"Bitcoin   : {ret_btc_2025:+.1f}%")
print(f"Alpha     : {ret_strat_2025 - ret_btc_2025:+.1f}% pts")

# 4. Plotting
fig = go.Figure()

# Strategy
fig.add_trace(go.Scatter(
    x=subset_2025.index, 
    y=subset_2025['strat_base100'], 
    name='Strategy (2025)', 
    line=dict(color='#00cc96', width=3)
))

# Bitcoin
fig.add_trace(go.Scatter(
    x=subset_2025.index, 
    y=subset_2025['btc_base100'], 
    name='Bitcoin (2025)', 
    line=dict(color='grey', dash='dot')
))

# Start line (0%)
fig.add_hline(y=1.0, line_color="white", line_width=1, opacity=0.5)

fig.update_layout(
    title='2025 Performance YTD (Strategy vs Bitcoin)', 
    template='plotly_dark',
    yaxis_title='Performance (1.20 = +20%)',
    yaxis_tickformat='.0%' # Percentage format
)

fig.show()

--- 2025 PERFORMANCE (YTD) ---
Strategy  : +29.6%
Bitcoin   : -10.6%
Alpha     : +40.2% pts
